# LangChain: Prompt Template

在 LangChain 中，Prompt Template 是一種設計和生成提示（Prompt）的方法，旨在使與模型的交互更具一致性和可重用性。Prompt Template 允許我們定義一個模板，並根據需要填充不同的變量或佔位符，從而生成動態的、定制化的提示。這樣的設計使得生成提示的過程更加高效，並能夠在多個不同的情境中重複使用。

## 什麼是 Prompt Template？
Prompt Template 是一個模板化的提示，設計來使模型的請求更一致、靈活和可控。通常，這些模板包含固定部分（如常見的指令或上下文）以及動態部分（如需要根據不同情境或用戶輸入來變化的內容）。

In [ ]:
!pip install --upgrade langchain
# 安裝langchain + google or openai
!pip install "langchain[google-genai]"
# !pip install "langchain[openai]"

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import (
    PromptTemplate, # 提示詞模板
    ChatPromptTemplate,
    FewShotPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from google.colab import userdata
from rich.pretty import pprint
import os

# 設定API Key
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# 建立模型
model = init_chat_model("google_genai:gemini-2.5-flash-lite")

### `PromptTemplate`

建立Prompt Template

In [ ]:
# 利用PromptTemplate.from_template建立
template = """生成{language}印出"hello world"的程式碼，使用繁體中文"""
prompt_template = PromptTemplate.from_template(template) # 模板內容
pprint(prompt_template)

In [ ]:
# 利用PromptTemplate初始化建立
prompt_template = PromptTemplate(
    template='生成{language}印出"hello world"的程式碼，使用繁體中文', # 模板內容
    input_variables=['language'], # 輸入變數名稱
)
pprint(prompt_template)

使用Prompt Template

In [ ]:
for lang in ['Python', 'Java', 'C++']:
    # 在模板中的language插入lang變數內容
    prompt = prompt_template.format(language=lang)
    print("prompt: ", prompt)
    # 呼叫llm回答
    print(model.invoke(prompt).content)
    print("===" * 10)

In [ ]:
# 多輸入參數 mutiple input parameters
multi_input_prompt = PromptTemplate(
    template='歡迎參與 {parm_adj} {param_topic} 課程。',
    input_variables=['parm_adj', 'param_topic']
)
prompt = multi_input_prompt.format(parm_adj='進階的', param_topic='LLM')
print(multi_input_prompt.template)
print(multi_input_prompt.input_variables)
# 填寫完結果
print(prompt)

In [ ]:
# from_template 不用設定 input_variables
multi_input_prompt = PromptTemplate.from_template('歡迎參與 {parm_adj} {param_topic} 課程。')
prompt = multi_input_prompt.format(parm_adj='進階的', param_topic='LLM')

print(multi_input_prompt.template)
print(multi_input_prompt.input_variables)
# 填寫完結果
print(prompt)

### `Partial prompt templates`

Partial prompt templates 允許開發者創建一個不完全的模板，並在後續的處理過程中進行動態填充或補充。這種方式非常適合用於多階段的生成過程，或者當你需要根據不同上下文來逐步擴展模板時。

In [ ]:
# 定義一個部分模板
template = "我的名字是{name}，我來自{location}。"
# 創建 PromptTemplate
partial_template = PromptTemplate.from_template(template)

# 部分填充模板
partial_filled = partial_template.partial(name="一隻羊")
pprint(partial_filled)  # 輸出：我的名字是一隻羊，我來自{location}。

# 填充剩餘變數
final_prompt = partial_filled.format(location="高雄")
print(final_prompt)

利用函式回傳值填充模板

In [ ]:
# 定義回傳當下日期的函式
def get_date():
    from datetime import datetime
    return datetime.now().strftime('%Y-%m-%d') # 2025-11-07

prompt_template = PromptTemplate.from_template(
    '你叫{name}，今天是{date}'
)
# 利用函式回傳值填充date內容
partial_prompt = prompt_template.partial(date=get_date)
pprint(partial_prompt)
filed_prompt = partial_prompt.format(name='一隻羊')
print(filed_prompt)

response = model.invoke([
    {"role": "system", "content": filed_prompt},
    {"role": "user", "content": "今天是幾月幾日？"},
])
pprint(response.text)

### `ChatPromptTemplate`

在 LangChain 中，ChatPromptTemplate 是專為處理聊天型應用設計的一種模板。它的目的是幫助開發者更輕鬆地創建和管理多輪對話中的提示（Prompts）。ChatPromptTemplate 使得在與用戶進行互動時，能夠根據對話的上下文動態生成和組合提示。

這種模板特別適用於基於聊天的應用，尤其是在需要多輪對話來構建模型回應時。與傳統的 PromptTemplate 類型相比，ChatPromptTemplate 能夠更好地處理模型與用戶的對話上下文，並且支持多角色的對話設計。

In [ ]:
# 定義一個聊天模板
chat_template = ChatPromptTemplate.from_messages([
    ("system", "你是一個叫做一隻羊的助手，請回答用戶的問題。"),
    ("system", "使用者習慣用繁體中文"),
    ("system", "使用者習慣用Python"),
    ("human", "{question}"),
])

# 格式化模板，將 {question} 變數填充
filled_prompt = chat_template.format(question="你叫做什麼名字？")
print(filled_prompt)

# 呼叫llm
pprint(model.invoke(filled_prompt).content)

## 練習請 AI 幫忙推薦旅遊景點

1. 建立 SystemMessage: 你是...
2. 建立 HumanMessage 模板（這裡使用變數country, season）
3. 利用 SystemMessage + HumanMessage 整合成ChatPromptTemplate
4. 使用partial設定country變數
5. 使用format設定season變數
6. 呼叫invoke印出結果

🧪 練習小挑戰
1. 換個主題：例如「咖啡推薦師」、「健身教練」、「電影評論家」

2. 結合 partial() 設定角色語氣或語言風格（例如：永遠用 Emoji 說話的 AI 🤖）

3. 將 season 改成 旅遊目的（如放鬆、美食、拍照）



In [ ]:
# 定義一個聊天模板
chat_template = ChatPromptTemplate.from_messages([
    ("system", "你是一個旅遊的助手"),
    ("human", "推薦3個{country} 在 {season} 旅遊景點")
])
pprint(chat_template)

chat_template = chat_template.partial(country="日本") # 先填國家
pprint(chat_template)

chat_template= chat_template.format(season="冬季") # 再填季節
print(chat_template)

# 呼叫llm
print(model.invoke(chat_template).content)

### `FewShotPromptTemplate`

在 LangChain 中，FewShotPromptTemplate 是一種專門用於處理少量示例學習（Few-Shot Learning）的模板類。少量示例學習是一種訓練方法，其中模型只需通過少量的範例來理解如何執行某個任務或生成預期的回應。在許多 NLP 任務中，少量示例學習能夠讓大語言模型在不需要大量數據的情況下，仍能有效地理解任務並給出合理的回應。

FewShotPromptTemplate 主要的目的是幫助開發者創建能夠在提示中包含少量範例的模板，並將這些範例與用戶的實際輸入一起傳遞給模型，從而實現少量學習。這樣，模型在生成回應時，會基於這些範例來推斷應該如何執行任務。

In [ ]:
print(model.invoke("3 🐑 4 = ? 繁體中文回答").content)

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate

example_prompt = PromptTemplate.from_template("問題: {question}\n答案：{answer}")

# 定義幾個範例
examples = [
    {"question": "5 🐑 3 = ？", "answer": "8"},
    {"question": "10 🐑 2 = ？", "answer": "12"},
    {"question": "11 🐑 8 = ？", "answer": "19"},
]
# 使用範例中的第0個問題和答案來生成相應的字串輸出
# `invoke(examples[0])` 根據範例來填充模板
# `to_string()` 會將生成的結果轉換為字串
print(example_prompt.invoke(examples[0]).to_string())

In [ ]:
# 創建 FewShotPromptTemplate
prompt = FewShotPromptTemplate(
    examples=examples, # 範例列表
    example_prompt=example_prompt, # 範例模板
    example_separator="\n",  # 範例之間的分隔符
    prefix="根據以下範例回答問題\n===", # prompt前綴
    suffix="Question: {input}", # prompt後綴
    input_variables=["input"],  # 這是需要填充的變數
)

# 格式化模板，將用戶的問題填入
prompt = prompt.format(input="15 🐑 2 等於多少？")
print(prompt, "\n--")

# 呼叫llm
response = model.invoke(prompt)
print(response.text)

🧠 練習主題：文句風格轉換（平鋪直敘 → 詩意風格）

🎯 目標：
學會使用 FewShotPromptTemplate 建立少量學習提示，給 AI 幾個例子後讓它模仿格式與風格

🧪 練習需求：
* 使用 FewShotPromptTemplate
* 設定幾個範例資料（examples）每個範例包含以下兩個key與對應文字
    * plain(平鋪直敘句子)
    * poetic(詩意風格句子)
* 建立 PromptTemplate
* 利用`PromptTemplate` + `examples`建立`FewShotPromptTemplate`
* 使用 .format() 將使用者的輸入加入few-shot prompt
* 利用llm回答